<a href="https://colab.research.google.com/github/dave7788/bdl-orm/blob/main/2301020056_UTS_SP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI DENGAN CERTAINTY FACTOR
# ============================================================================

import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print(" SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI ".center(80, "="))
print(" Menggunakan Metode Certainty Factor (CF) ".center(80, "="))
print("="*80)
print(f"Waktu Eksekusi: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

# ============================================================================
# 1. KNOWLEDGE BASE
# ============================================================================

print("\n" + "="*80)
print(" KNOWLEDGE BASE ".center(80, "="))
print("="*80)

# Daftar Penyakit
penyakit_list = {
    'P1': 'Hawar Daun Bakteri',
    'P2': 'Blas',
    'P3': 'Busuk Batang',
    'P4': 'Tungro'
}

# Daftar Gejala
gejala_list = {
    'G1': 'Daun menguning',
    'G2': 'Bercak coklat pada daun',
    'G3': 'Daun mengering',
    'G4': 'Batang busuk dan berbau',
    'G5': 'Pertumbuhan terhambat',
    'G6': 'Daun berlubang',
    'G7': 'Daun berkerut',
    'G8': 'Tanaman kerdil'
}

# Rules dengan CF Pakar (14 rules total)
rules = {
    'P1': [  # Hawar Daun Bakteri
        {'gejala': 'G1', 'cf_pakar': 0.6},
        {'gejala': 'G2', 'cf_pakar': 0.8},
        {'gejala': 'G6', 'cf_pakar': 0.7}
    ],
    'P2': [  # Blas
        {'gejala': 'G2', 'cf_pakar': 0.7},
        {'gejala': 'G3', 'cf_pakar': 0.8},
        {'gejala': 'G5', 'cf_pakar': 0.6}
    ],
    'P3': [  # Busuk Batang
        {'gejala': 'G4', 'cf_pakar': 0.9},
        {'gejala': 'G5', 'cf_pakar': 0.7},
        {'gejala': 'G1', 'cf_pakar': 0.5}
    ],
    'P4': [  # Tungro
        {'gejala': 'G1', 'cf_pakar': 0.8},
        {'gejala': 'G7', 'cf_pakar': 0.9},
        {'gejala': 'G8', 'cf_pakar': 0.85}
    ]
}

print("\nDaftar Penyakit:")
for kode, nama in penyakit_list.items():
    print(f"  • {kode}: {nama}")

print("\nDaftar Gejala:")
for kode, nama in gejala_list.items():
    print(f"  • {kode}: {nama}")

print("\nRules (Total: 14 rules):")
rule_count = 0
for penyakit_id, rule_list in rules.items():
    print(f"\n  {penyakit_list[penyakit_id]}:")
    for rule in rule_list:
        rule_count += 1
        print(f"    Rule {rule_count}: IF {rule['gejala']} ({gejala_list[rule['gejala']]}) THEN {penyakit_id} (CF = {rule['cf_pakar']})")

# ============================================================================
# 2. FUNGSI PERHITUNGAN CERTAINTY FACTOR
# ============================================================================

def hitung_cf_kombinasi(cf_old, cf_new):
    """
    Menghitung CF kombinasi untuk multiple evidence
    Formula: CF_kombinasi = CF_old + CF_new × (1 - CF_old)
    """
    return cf_old + cf_new * (1 - cf_old)

def hitung_cf_he(cf_user, cf_pakar):
    """
    Menghitung CF(H,E) = CF(user) × CF(pakar)
    """
    return cf_user * cf_pakar

def validasi_cf_input(nilai):
    """
    Validasi input CF user (harus antara 0-1)
    """
    try:
        cf = float(nilai)
        if 0 <= cf <= 1:
            return True, cf
        else:
            return False, None
    except:
        return False, None

# ============================================================================
# 3. FORWARD CHAINING ENGINE
# ============================================================================

def forward_chaining(gejala_user):
    """
    Implementasi forward chaining untuk diagnosa penyakit

    Input:
        gejala_user = {'G1': 0.8, 'G2': 0.6, ...}

    Output:
        hasil_diagnosa = {'P1': 0.75, 'P2': 0.62, ...}
    """
    hasil_diagnosa = {}

    print("\n" + "="*80)
    print(" PROSES FORWARD CHAINING ".center(80, "="))
    print("="*80)

    # Iterasi untuk setiap penyakit
    for penyakit_id, rule_list in rules.items():
        print(f"\n{'-'*80}")
        print(f"Evaluasi Penyakit: {penyakit_list[penyakit_id]} ({penyakit_id})")
        print(f"{'-'*80}")

        cf_penyakit = 0  # CF awal untuk penyakit ini
        evidence_count = 0

        # Evaluasi setiap rule untuk penyakit ini
        for rule in rule_list:
            gejala_id = rule['gejala']
            cf_pakar = rule['cf_pakar']

            # Cek apakah gejala ada di input user
            if gejala_id in gejala_user:
                cf_user = gejala_user[gejala_id]

                # Hitung CF(H,E) = CF(user) × CF(pakar)
                cf_he = hitung_cf_he(cf_user, cf_pakar)

                print(f"\n  Rule: IF {gejala_id} ({gejala_list[gejala_id]}) THEN {penyakit_id}")
                print(f"    • CF User  : {cf_user:.2f}")
                print(f"    • CF Pakar : {cf_pakar:.2f}")
                print(f"    • CF(H,E)  : {cf_user:.2f} × {cf_pakar:.2f} = {cf_he:.4f}")

                # Kombinasikan dengan CF sebelumnya
                if evidence_count == 0:
                    cf_penyakit = cf_he
                    print(f"    • CF Penyakit: {cf_penyakit:.4f} (evidence pertama)")
                else:
                    cf_old = cf_penyakit
                    cf_penyakit = hitung_cf_kombinasi(cf_old, cf_he)
                    print(f"    • CF Kombinasi: {cf_old:.4f} + {cf_he:.4f} × (1 - {cf_old:.4f})")
                    print(f"    • CF Penyakit: {cf_penyakit:.4f}")

                evidence_count += 1
            else:
                print(f"\n  Rule: IF {gejala_id} ({gejala_list[gejala_id]}) THEN {penyakit_id}")
                print(f"    • Gejala tidak ditemukan pada input user (SKIP)")

        # Simpan hasil CF untuk penyakit ini
        if cf_penyakit > 0:
            hasil_diagnosa[penyakit_id] = cf_penyakit
            print(f"\n  ✓ CF Final {penyakit_list[penyakit_id]}: {cf_penyakit:.4f}")
        else:
            print(f"\n  ✗ Tidak ada evidence untuk {penyakit_list[penyakit_id]}")

    return hasil_diagnosa

# ============================================================================
# 4. FUNGSI TAMPILAN HASIL
# ============================================================================

def tampilkan_hasil(hasil_diagnosa, gejala_user):
    """
    Menampilkan hasil diagnosa dengan ranking dan rekomendasi
    """
    print("\n" + "="*80)
    print(" HASIL DIAGNOSA ".center(80, "="))
    print("="*80)

    # Tampilkan gejala yang diinput
    print("\n📋 Gejala yang Dialami:")
    for gejala_id, cf_user in gejala_user.items():
        print(f"  • {gejala_id} ({gejala_list[gejala_id]}): CF User = {cf_user:.2f}")

    # Ranking penyakit berdasarkan CF tertinggi
    if hasil_diagnosa:
        print("\n" + "─"*80)
        print(" RANKING PENYAKIT (CF > 0) ".center(80, "─"))
        print("─"*80)

        # Sort berdasarkan CF tertinggi
        ranking = sorted(hasil_diagnosa.items(), key=lambda x: x[1], reverse=True)

        print(f"\n{'Rank':<6} {'Kode':<8} {'Nama Penyakit':<30} {'CF':<15} {'Persentase':<15}")
        print("─"*80)

        for rank, (penyakit_id, cf_value) in enumerate(ranking, 1):
            persentase = cf_value * 100
            print(f"{rank:<6} {penyakit_id:<8} {penyakit_list[penyakit_id]:<30} {cf_value:.4f}{'':<9} {persentase:.2f}%")

        # Kesimpulan Diagnosa
        print("\n" + "="*80)
        print(" KESIMPULAN DIAGNOSA ".center(80, "="))
        print("="*80)

        diagnosa_utama = ranking[0]
        penyakit_id = diagnosa_utama[0]
        cf_value = diagnosa_utama[1]
        persentase = cf_value * 100

        if cf_value >= 0.8:
            tingkat_keyakinan = "SANGAT TINGGI"
            emoji = "🔴"
        elif cf_value >= 0.6:
            tingkat_keyakinan = "TINGGI"
            emoji = "🟠"
        elif cf_value >= 0.4:
            tingkat_keyakinan = "SEDANG"
            emoji = "🟡"
        else:
            tingkat_keyakinan = "RENDAH"
            emoji = "🟢"

        print(f"\n{emoji} Diagnosa Utama: {penyakit_list[penyakit_id]}")
        print(f"   Certainty Factor: {cf_value:.4f} ({persentase:.2f}%)")
        print(f"   Tingkat Keyakinan: {tingkat_keyakinan}")

        # Rekomendasi berdasarkan penyakit
        print("\n" + "─"*80)
        print(" REKOMENDASI TINDAKAN ".center(80, "─"))
        print("─"*80)

        rekomendasi = {
            'P1': [
                "1. Buang dan musnahkan bagian tanaman yang terinfeksi",
                "2. Aplikasikan bakterisida berbahan aktif tembaga",
                "3. Perbaiki drainase untuk mengurangi kelembaban",
                "4. Gunakan varietas tahan penyakit untuk penanaman berikutnya"
            ],
            'P2': [
                "1. Gunakan fungisida sistemik seperti trikasiklazol atau isoprotiolan",
                "2. Potong dan musnahkan bagian tanaman yang terinfeksi",
                "3. Atur jarak tanam untuk sirkulasi udara yang baik",
                "4. Kurangi pemberian nitrogen berlebih"
            ],
            'P3': [
                "1. Buang dan bakar tanaman yang terinfeksi parah",
                "2. Aplikasikan fungisida berbahan aktif validamycin",
                "3. Perbaiki drainase dan hindari genangan air",
                "4. Rotasi tanaman dengan tanaman non-padi"
            ],
            'P4': [
                "1. Kendalikan vektor wereng hijau dengan insektisida",
                "2. Cabut dan musnahkan tanaman yang terinfeksi",
                "3. Gunakan varietas padi tahan tungro",
                "4. Hindari penanaman padi secara berturut-turut"
            ]
        }

        print()
        for item in rekomendasi[penyakit_id]:
            print(f"  {item}")

        # Penyakit lain yang mungkin
        if len(ranking) > 1:
            print("\n" + "─"*80)
            print(" PENYAKIT LAIN YANG MUNGKIN ".center(80, "─"))
            print("─"*80)
            print()
            for rank, (p_id, cf_val) in enumerate(ranking[1:], 2):
                print(f"  {rank}. {penyakit_list[p_id]} (CF: {cf_val:.4f} / {cf_val*100:.2f}%)")
    else:
        print("\n⚠️  Tidak ada penyakit yang terdiagnosa.")
        print("    Gejala yang diinput tidak cukup untuk mendiagnosa penyakit.")
        print("    Silakan periksa kembali gejala atau konsultasi dengan ahli pertanian.")

# ============================================================================
# 5. FUNGSI INPUT USER (CLI)
# ============================================================================

def input_gejala_manual():
    """
    Interface CLI untuk input gejala dari user
    """
    print("\n" + "="*80)
    print(" INPUT GEJALA ".center(80, "="))
    print("="*80)

    print("\nDaftar Gejala yang Tersedia:")
    for gejala_id, nama_gejala in gejala_list.items():
        print(f"  {gejala_id}: {nama_gejala}")

    print("\n" + "─"*80)
    print("Masukkan gejala yang dialami tanaman padi Anda.")
    print("Format: [Kode Gejala] [CF User (0-1)]")
    print("Contoh: G1 0.8")
    print("Ketik 'selesai' jika sudah selesai input.")
    print(f"Maksimal input: {len(gejala_list)} gejala")
    print("─"*80)

    gejala_user = {}

    while len(gejala_user) < len(gejala_list):
        print(f"\nInput gejala ke-{len(gejala_user)+1} (atau ketik 'selesai' untuk berhenti):")
        user_input = input("  >> ").strip()

        if user_input.lower() == 'selesai':
            if len(gejala_user) == 0:
                print("  ⚠️  Anda belum menginput gejala apapun!")
                continue
            break

        # Parse input
        parts = user_input.split()
        if len(parts) != 2:
            print("  ❌ Format salah! Gunakan format: [Kode Gejala] [CF User]")
            continue

        gejala_id = parts[0].upper()
        cf_input = parts[1]

        # Validasi gejala
        if gejala_id not in gejala_list:
            print(f"  ❌ Gejala {gejala_id} tidak ditemukan!")
            continue

        # Cek apakah gejala sudah diinput
        if gejala_id in gejala_user:
            print(f"  ⚠️  Gejala {gejala_id} sudah diinput sebelumnya!")
            continue

        # Validasi CF
        valid, cf_value = validasi_cf_input(cf_input)
        if not valid:
            print(f"  ❌ CF harus berupa angka antara 0 dan 1!")
            continue

        # Simpan gejala
        gejala_user[gejala_id] = cf_value
        print(f"  ✓ Gejala {gejala_id} ({gejala_list[gejala_id]}) dengan CF {cf_value:.2f} berhasil ditambahkan!")

        # Jika sudah mencapai 8 gejala, otomatis berhenti
        if len(gejala_user) == len(gejala_list):
            print(f"\n  ✓ Semua {len(gejala_list)} gejala telah diinput. Input selesai.")
            break

    return gejala_user

# ============================================================================
# 6. TESTING DENGAN 3 TEST CASES
# ============================================================================

print("\n" + "="*80)
print(" TESTING DENGAN 3 TEST CASES ".center(80, "="))
print("="*80)

# Test Case 1: Hawar Daun Bakteri (Dominan)
print("\n" + "█"*80)
print(" TEST CASE 1: HAWAR DAUN BAKTERI ".center(80, "█"))
print("█"*80)

test_case_1 = {
    'G1': 0.8,  # Daun menguning
    'G2': 0.9,  # Bercak coklat pada daun
    'G6': 0.7   # Daun berlubang
}

print("\nSkenario: Tanaman padi menunjukkan gejala kuat Hawar Daun Bakteri")
hasil_1 = forward_chaining(test_case_1)
tampilkan_hasil(hasil_1, test_case_1)

# Test Case 2: Multiple Diseases (Tungro + Busuk Batang)
print("\n\n" + "█"*80)
print(" TEST CASE 2: KONDISI MIXED (TUNGRO + BUSUK BATANG) ".center(80, "█"))
print("█"*80)

test_case_2 = {
    'G1': 0.9,  # Daun menguning (P1, P3, P4)
    'G4': 0.8,  # Batang busuk dan berbau (P3)
    'G7': 0.7,  # Daun berkerut (P4)
    'G8': 0.6   # Tanaman kerdil (P4)
}

print("\nSkenario: Tanaman menunjukkan gejala campuran beberapa penyakit")
hasil_2 = forward_chaining(test_case_2)
tampilkan_hasil(hasil_2, test_case_2)

# Test Case 3: Blas (Sedang)
print("\n\n" + "█"*80)
print(" TEST CASE 3: BLAS (TINGKAT SEDANG) ".center(80, "█"))
print("█"*80)

test_case_3 = {
    'G2': 0.6,  # Bercak coklat pada daun (P1, P2)
    'G3': 0.7,  # Daun mengering (P2)
    'G5': 0.5   # Pertumbuhan terhambat (P2, P3)
}

print("\nSkenario: Gejala mengarah ke penyakit Blas dengan tingkat keyakinan sedang")
hasil_3 = forward_chaining(test_case_3)
tampilkan_hasil(hasil_3, test_case_3)

# ============================================================================
# 7. MODE INTERAKTIF (OPSIONAL)
# ============================================================================

print("\n\n" + "="*80)
print(" MODE INTERAKTIF ".center(80, "="))
print("="*80)

print("\nApakah Anda ingin mencoba diagnosa dengan input manual?")
print("Ketik 'ya' untuk melanjutkan, atau 'tidak' untuk selesai.")

pilihan = input("\n>> ").strip().lower()

if pilihan == 'ya':
    gejala_user = input_gejala_manual()

    print("\n" + "█"*80)
    print(" HASIL DIAGNOSA USER INPUT ".center(80, "█"))
    print("█"*80)

    hasil_user = forward_chaining(gejala_user)
    tampilkan_hasil(hasil_user, gejala_user)

# ============================================================================
# PROGRAM SELESAI
# ============================================================================

print("\n" + "="*80)
print(" PROGRAM SELESAI ".center(80, "="))
print("="*80)

print("\n✅ Sistem Pakar Diagnosa Penyakit Tanaman Padi")
print("✅ Metode: Certainty Factor (CF) dengan Forward Chaining")
print("✅ Total Penyakit: 4 penyakit")
print("✅ Total Gejala: 8 gejala")
print("✅ Total Rules: 14 rules")
print("✅ Test Cases: 3 skenario berbeda")
print("\n" + "="*80)

================= SISTEM PAKAR DIAGNOSA PENYAKIT TANAMAN PADI ==================
=================== Menggunakan Metode Certainty Factor (CF) ===================
Waktu Eksekusi: 2025-10-30 01:04:23

================================ KNOWLEDGE BASE ================================

Daftar Penyakit:
  • P1: Hawar Daun Bakteri
  • P2: Blas
  • P3: Busuk Batang
  • P4: Tungro

Daftar Gejala:
  • G1: Daun menguning
  • G2: Bercak coklat pada daun
  • G3: Daun mengering
  • G4: Batang busuk dan berbau
  • G5: Pertumbuhan terhambat
  • G6: Daun berlubang
  • G7: Daun berkerut
  • G8: Tanaman kerdil

Rules (Total: 14 rules):

  Hawar Daun Bakteri:
    Rule 1: IF G1 (Daun menguning) THEN P1 (CF = 0.6)
    Rule 2: IF G2 (Bercak coklat pada daun) THEN P1 (CF = 0.8)
    Rule 3: IF G6 (Daun berlubang) THEN P1 (CF = 0.7)

  Blas:
    Rule 4: IF G2 (Bercak coklat pada daun) THEN P2 (CF = 0.7)
    Rule 5: IF G3 (Daun mengering) THEN P2 (CF = 0.8)
    Rule 6: IF G5 (Pertumbuhan terhambat) THEN P2 (CF = 